## Imports

In [9]:
import numpy as np
import time
import random
from tqdm import tqdm
import os
from PIL import Image
import cv2
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from gym import spaces
import pygame

import os,sys

sys.path.append(os.path.join(sys.path[0], ".."))

import matplotlib.pylab as plt
from IPython import display
%matplotlib inline

from environment.GridEnv import GridWorld
from utils import run_agent_in_env

from stable_baselines3 import DQN, PPO, A2C


### Spawning windows with WSL setup

To be able to deploy window with pygame in WSL follow this [tutorial](https://medium.com/javarevisited/using-wsl-2-with-x-server-linux-on-windows-a372263533c3)

##### TL;DR

- Install x11 server
- In WSL setup Display config: `DISPLAY=$(cat /etc/resolv.conf | grep nameserver | awk '{print $2}'):0`
- Run x11 server with command like: `C:\Program Files\VcXsrv\vcxsrv.exe" :0 -ac -terminate -lesspointer -multiwindow -clipboard -wgl -dpi auto`


## Training

### Initial training

In [ ]:
GRID_SIDE_LENGTH=4

env = GridWorld(GRID_SIDE_LENGTH)
env.reset()
model = A2C('MlpPolicy', env, verbose=1) #Also tried with DQN and PPO

models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

TIMESTEPS = 200000
iters = 1
while True:
    iters += 1
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
    model.save(f"{models_dir}/2_{TIMESTEPS*iters}")

### Re-training (bigger grids)

In [ ]:
model = A2C.load(f"models/1650141014/2_750000.zip")

GRID_SIDE_LENGTH=12 #Previously 8 and 10

env = GridWorld(GRID_SIDE_LENGTH)
env.reset()

model.set_env(env)

models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

TIMESTEPS = 200000
iters = 1
while True:
    iters += 1
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"./PPO")
    model.save(f"{models_dir}/2_{TIMESTEPS*iters}")

### Test the agent - run rendered simulation

In [ ]:
GRID_SIDE_LENGTH=12

env = GridWorld(GRID_SIDE_LENGTH)
env.reset()

model= A2C.load(f"models/1650180249/2_9200000.zip")
rewards = run_agent_in_env(env, model, 5, learning=False, plot=False, render=True, plot_interval=20)

### Extras/utils

In [ ]:
# SAVE/ LOAD MODEL TO / FROM FILE
T.save(agent.q_eval, f"./model_{GRID_SIDE_LENGTH}_{EPISODES}")
model=T.load(f"./model_{GRID_SIDE_LENGTH}_1000")

In [ ]:
# Remove hangin pygame window
pygame.quit()